<a href="https://colab.research.google.com/github/Eindveer/Audio-Video-Analyzer/blob/main/File_Scanner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q openai-whisper transformers librosa pandas openpyxl moviepy \
  keybert rake-nltk spacy pyannote.audio torch torchvision torchaudio \
  accelerate

import nltk
nltk.download('stopwords')

!python -m spacy download en_core_web_sm


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 23.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [13]:
from google.colab import files
uploaded = files.upload()
file_path = next(iter(uploaded))


Saving Interview-reduced_size,_Bobbie_R._Allen_at_Staff_Breakfast_Radio_Show,_Pampa_Texas,_1971.wav.mp3 to Interview-reduced_size,_Bobbie_R._Allen_at_Staff_Breakfast_Radio_Show,_Pampa_Texas,_1971.wav.mp3


In [14]:
from moviepy.editor import VideoFileClip
import os

def extract_audio(file_path):
    if file_path.lower().endswith(('.mp4', '.mov', '.mkv', '.avi')):
        clip = VideoFileClip(file_path)
        audio_path = "extracted_audio.wav"
        clip.audio.write_audiofile(audio_path)
        return audio_path
    return file_path

audio_file = extract_audio(file_path)


In [15]:
import whisper

model = whisper.load_model("tiny")
result = model.transcribe(audio_file, task="transcribe")
transcript = result["text"]
segments = result["segments"]



  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



In [16]:
from transformers import pipeline

summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

def summarize_text(text, max_chunk=500):
    sentences = text.split(". ")
    chunks, chunk = [], ""
    for sentence in sentences:
        if len(chunk) + len(sentence) <= max_chunk:
            chunk += sentence + ". "
        else:
            chunks.append(chunk.strip())
            chunk = sentence + ". "
    chunks.append(chunk.strip())
    summary = ""
    for chunk in chunks:
        summary += summarizer(chunk, max_length=100, min_length=30, do_sample=False)[0]['summary_text'] + " "
    return summary.strip()

summary = summarize_text(transcript)


Device set to use cpu
Your max_length is set to 100, but your input_length is only 92. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=46)
Your max_length is set to 100, but your input_length is only 85. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=42)
Your max_length is set to 100, but your input_length is only 97. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=48)
Your max_length is set to 100, but your input_length is only 87. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...

In [17]:
from keybert import KeyBERT
kw_model = KeyBERT()
keywords = kw_model.extract_keywords(transcript, keyphrase_ngram_range=(1, 2), stop_words='english', top_n=10)
keywords = [kw[0] for kw in keywords]


In [18]:
from pyannote.audio import Pipeline

pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization",
    use_auth_token="hugging face key"
)

diarization = pipeline(audio_file)

for turn, _, speaker in diarization.itertracks(yield_label=True):
    print(f"{turn.start:.1f}s - {turn.end:.1f}s: {speaker}")


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/pyannote/models--pyannote--segmentation/snapshots/c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.5.1+cu124. Bad things might happen unless you revert torch to 1.x.


  warnings.warn(



0.0s - 10.3s: SPEAKER_01
9.4s - 23.8s: SPEAKER_00
24.5s - 32.4s: SPEAKER_00
33.2s - 48.7s: SPEAKER_00
49.4s - 55.9s: SPEAKER_00
56.8s - 64.1s: SPEAKER_00
65.0s - 67.5s: SPEAKER_00
68.6s - 77.9s: SPEAKER_00
80.5s - 85.1s: SPEAKER_01
84.5s - 99.8s: SPEAKER_00
100.7s - 111.5s: SPEAKER_01
111.5s - 131.7s: SPEAKER_00
132.5s - 176.4s: SPEAKER_00
177.3s - 180.5s: SPEAKER_00
180.5s - 188.0s: SPEAKER_01
188.0s - 212.5s: SPEAKER_00


In [19]:
print("Timestamped Segments:")
for seg in segments:
    print(f"{seg['start']:.2f}s - {seg['end']:.2f}s: {seg['text']}")


Timestamped Segments:
0.00s - 9.00s:  you're saying that there's not a great deal of possibility of midair crashes. Of course it exists, but the possibility is fairly remote isn't not.
9.00s - 17.00s:  Well, I started speech one time and said that we had the potential or the problem of midair collisions.
17.00s - 21.00s:  There was since Wilbert turned or Wilbert said, let's build another word.
22.00s - 33.00s:  I think that right now the midair collision problem is being discussed throughout the entire United States.
33.00s - 46.00s:  I think that it can't help but the problem can't help but increase because our civil aviation fleet is projected to increase to approximately 260,000 airplanes in the next 10 years.
46.00s - 56.00s:  Now that's doubling the fleet that we have today. Right now we have approximately 120 to 130,000 airplanes in the civil aviation fleet.
56.00s - 65.00s:  And out of this 130,000 aircraft approximately 27 to 2800 are involved in commercial aviation.
65.00s - 

In [20]:
import spacy
nlp = spacy.load("en_core_web_sm")
doc = nlp(transcript)

named_entities = [(ent.text, ent.label_) for ent in doc.ents]


In [21]:
from transformers import pipeline

emotion_model = pipeline("text-classification", model="nateraw/bert-base-uncased-emotion")
emotions = emotion_model(transcript[:512])



Device set to use cpu


In [22]:
import pandas as pd


with open("summary.txt", "w") as f:
    f.write(summary)


df = pd.DataFrame({
    "Top Keywords": [", ".join(keywords)],
    "Named Entities": [", ".join([e[0] for e in named_entities])],
    "Emotions": [", ".join([f"{e['label']} ({e['score']:.2f})" for e in emotions])]
})

df.to_excel("analysis_report.xlsx", index=False)


files.download("summary.txt")
files.download("analysis_report.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>